# Fine Tunning using unsloth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# Verificar se CUDA está disponível para acelerar o processamento
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

CUDA Available:  True
CUDA Device Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU
Usando dispositivo: cuda


# Define Model

In [ ]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
initial_model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="unsloth/Llama-3.2-3B-bnb-4bit",
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # device_map="auto"
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = initial_model


==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
initial_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (

# Dataset TeleQnA

## Release 17 Questions

In [ ]:
import json

# Path to the TeleQnA processed question in JSON file
rel17_questions_path = r"../Files/rel17_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_questions_path, "r", encoding="utf-8") as file:
    rel17_questions = json.load(file)
print(len(rel17_questions))

In [ ]:
# Path to the TeleQnA processed question in JSON file
rel17_200_questions_path = r"../Files/rel17_100_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_200_questions_path, "r", encoding="utf-8") as file:
    rel17_200_questions = json.load(file)
print(len(rel17_200_questions))

In [ ]:
rel17_other_questions = [q for q in rel17_questions if q not in rel17_100_questions]
print(len(rel17_other_questions))

rel17_other_questions_length = 500

rel17_other_questions = rel17_other_questions[:rel17_other_questions_length]


In [ ]:
rel17_other_questions[0]

## Questions without rel 17 and 18

In [ ]:
# Path to the TeleQnA processed question in JSON file
questions_no_rel_17_18_path = r"../Files/questions_no_rel_17_18.json"

# Load the TeleQnA data just release 17
with open(questions_no_rel_17_18_path, "r", encoding="utf-8") as file:
    questions_no_rel_17_18 = json.load(file)
print(len(questions_no_rel_17_18))

In [ ]:
questions_no_rel_17_18_length = 1500
questions_no_rel_17_18 = questions_no_rel_17_18[:questions_no_rel_17_18_length]
print(len(questions_no_rel_17_18))

In [ ]:
questions_no_rel_17_18[0]

## Train Data

In [ ]:
# train_questions = rel17_other_questions
train_questions = rel17_other_questions + questions_no_rel_17_18
print(len(train_questions))

In [ ]:
train_questions[0]

In [ ]:
from datasets import Dataset

# Structure to store pairs of questions and explanations
data = []

half_questions = len(train_questions)//2

# Fill the dataset with (question, explanation) pairs
for item in train_questions[:half_questions]:

    human_value = (
        f"{item['question']}"
    )

    # Combine the answer and explanation
    gpt_value = (
        f"{item['explanation']}"
    )

    # Create a dictionary for each input pair
    pair = [
        {'from': 'human', 'value': human_value},  # For the question
        {'from': 'gpt', 'value': gpt_value}  # For the explanation
    ]

    data.append(pair)  # Add the pair to the dataset

data_no_options = data
print(data_no_options[0])

In [ ]:
from datasets import Dataset

# Structure to store pairs of questions, options, answers, and explanations
data = []

# Fill the dataset with (question + options, answer + explanation) pairs
for item in train_questions[half_questions:]:

    # Extract options
    options = [f"{key}: {value}" for key, value in item.items() if 'option' in key]
    # Combine the question and options
    human_value = (
        f"Question: {item['question']}\n"
        f"Options:\n" + "\n".join(options) + "\n"
    )

    # Combine the answer and explanation
    gpt_value = (
        f"Answer: {item['answer']}\n"
        f"Explanation: {item['explanation']}"
    )

    # Create a dictionary for each input pair
    pair = [
        {'from': 'human', 'value': human_value},  # Question with options
        {'from': 'gpt', 'value': gpt_value}       # Answer with explanation
    ]

    data.append(pair)  # Add the pair to the dataset

# Create the dataset using Hugging Face
data_options = data
print(data_options[0])

In [ ]:
import random

data_total = data_no_options + data_options
# Shuffle the combined data
random.shuffle(data_total)
print(len(data_total))

In [ ]:
# Create the dataset using Hugging Face
# Convert the list of pairs into the appropriate format
formatted_data = {'conversations': data_total}

# Transform the data into a Dataset
dataset = Dataset.from_dict(formatted_data)

# Display information about the created dataset
print(dataset)

# Display the first entry of the dataset
print(dataset[0])

In [ ]:
# dataset.save_to_disk('../Files/train_questions_dataset_2000_questions')

In [ ]:
from datasets import load_from_disk

dataset_path = '/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Files/train_questions_dataset_4000_questions'

dataset = load_from_disk(dataset_path)

print(len(dataset))

4000


In [ ]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'What is the purpose of the rotation matrix R in the LCS to GCS transformation? [3GPP Release 17]'},
  {'from': 'gpt',
   'value': 'The rotation matrix R is used to transform the field components from the LCS to the GCS.'}]}

In [ ]:
dataset[5]

{'conversations': [{'from': 'human',
   'value': 'Question: What is one disadvantage of performing a CMA (Cloud-based Mobile Augmentation) process on mobile computing?\nOptions:\noption 1: Dependency on high performance networking infrastructure\noption 2: Excessive communication overhead and traffic\noption 3: Unauthorized access to offloaded data\noption 4: Application development complexity\noption 5: All of the above\n'},
  {'from': 'gpt',
   'value': 'Answer: option 5: All of the above\nExplanation: Performing a CMA process on mobile computing can have disadvantages such as dependency on high performance networking infrastructure, excessive communication overhead and traffic, unauthorized access to offloaded data, and application development complexity.'}]}

# Build TeleQnA Dataset for training

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 4000
})

In [ ]:
dataset[0]['text']

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the purpose of the rotation matrix R in the LCS to GCS transformation? [3GPP Release 17]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe rotation matrix R is used to transform the field components from the LCS to the GCS.<|eot_id|>'

# Training

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# del model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 300,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/4000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the purpose of the rotation matrix R in the LCS to GCS transformation? [3GPP Release 17]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe rotation matrix R is used to transform the field components from the LCS to the GCS.<|eot_id|>'

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]])

'                                                           \n\nThe rotation matrix R is used to transform the field components from the LCS to the GCS.<|eot_id|>'

## Show current memory stats


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.688 GB of memory reserved.


## Initial Loss

In [ ]:
trainer.train_dataset[0].items()

dict_items([('input_ids', [128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 5887, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 3923, 374, 279, 7580, 315, 279, 12984, 6303, 432, 304, 279, 84968, 311, 480, 6546, 18475, 30, 510, 18, 38, 4505, 17836, 220, 1114, 60, 128009, 128006, 78191, 128007, 271, 791, 12984, 6303, 432, 374, 1511, 311, 5276, 279, 2115, 6956, 505, 279, 84968, 311, 279, 480, 6546, 13, 128009]), ('attention_mask', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), ('labels', [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Configura o collator e DataLoader
collator = DataCollatorForSeq2Seq(tokenizer)
loader = DataLoader(trainer.train_dataset,
                    batch_size=2,  # Tamanho do batch escolhido
                    collate_fn=collator,
                    num_workers=2)

# Variáveis para armazenar a loss total e o número de exemplos
total_loss = 0.0
num_batches = 0

# Coloca o modelo em modo de avaliação
model.eval()

# Desativa o cálculo de gradiente para economizar memória
with torch.no_grad():
    for batch in tqdm(loader, desc="Calculating initial loss"):
        # Move o batch para a GPU (se disponível)
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Acumula a loss
        total_loss += outputs.loss.item()
        num_batches += 1

# Calcula a loss média
average_loss = total_loss / num_batches
print(f"Initial mean loss: {average_loss}")


Calculating initial loss: 100%|██████████| 2000/2000 [04:36<00:00,  7.23it/s]

Initial mean loss: 1.7166468909233807


## Training Session

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,000 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 16
\        /    Total batch size = 32 | Total steps = 300
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.695400
2,1.616000
3,1.292700
4,1.252200
5,1.438800
6,1.112600
7,1.070800
8,0.860700
9,1.258400
10,1.180300


## Show final memory and time stats

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2353.7029 seconds used for training.
39.23 minutes used for training.
Peak reserved memory = 8.574 GB.
Peak reserved memory for training = 1.886 GB.
Peak reserved memory % of max memory = 58.137 %.
Peak reserved memory for training % of max memory = 12.788 %.


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
                (b

## Loss Pos training

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Configure the collator and DataLoader
collator = DataCollatorForSeq2Seq(tokenizer)
loader = DataLoader(
    trainer.train_dataset,
    batch_size=2,  # Chosen batch size
    collate_fn=collator,
    num_workers=2
)

# Variables to store the total loss and the number of valid batches
total_loss = 0.0
num_batches = 0

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation to save memory
with torch.no_grad():
    for batch in tqdm(loader, desc="Calculating loss post-training"):
        # Move the batch to GPU (if available)
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Check if the loss is NaN
        if not torch.isnan(outputs.loss):
            total_loss += outputs.loss.item()
            num_batches += 1
            # print(outputs.loss.item())

print(f"Number of valid batches: {num_batches}")

# Calculate the average loss
if num_batches > 0:
    average_loss = total_loss / num_batches
    print(f"Post-training mean loss: {average_loss}")
else:
    print("No valid batches found.")


Calculating loss post-training: 100%|██████████| 2000/2000 [04:37<00:00,  7.22it/s]

Number of valid batches: 2000
Post-training mean loss: 0.6859039425058291


# Inference

## Question with option

In [ ]:
dataset[5]['conversations'][0]

{'from': 'human',
 'value': 'Question: What is one disadvantage of performing a CMA (Cloud-based Mobile Augmentation) process on mobile computing?\nOptions:\noption 1: Dependency on high performance networking infrastructure\noption 2: Excessive communication overhead and traffic\noption 3: Unauthorized access to offloaded data\noption 4: Application development complexity\noption 5: All of the above\n'}

In [ ]:
question = dataset[5]['conversations'][0]['value']

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: What is one disadvantage of performing a CMA (Cloud-based Mobile Augmentation) process on mobile computing?\nOptions:\noption 1: Dependency on high performance networking infrastructure\noption 2: Excessive communication overhead and traffic\noption 3: Unauthorized access to offloaded data\noption 4: Application development complexity\noption 5: All of the above\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer: option 5: All of the above\nExplanation: CMA process has limitations like dependency on high performance networking infrastructure, excessive communication overhead, unauthorized access to offloaded data, and increased application development complexity.<|eot_id|>']

## Question with no option

In [ ]:
dataset[0]['conversations'][0]['value']

'What is the purpose of the rotation matrix R in the LCS to GCS transformation? [3GPP Release 17]'

In [ ]:
question = dataset[0]['conversations'][0]['value']

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the purpose of the rotation matrix R in the LCS to GCS transformation? [3GPP Release 17]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe purpose of the rotation matrix R in the LCS to GCS transformation is to transform the spatial coordinates into Earth-fixed coordinates.<|eot_id|>']

# Save model

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions", safe_serialization=False)
tokenizer.save_pretrained("/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions")

('/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions/tokenizer_config.json',
 '/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions/special_tokens_map.json',
 '/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions/tokenizer.json')

In [ ]:
# model.save_pretrained_merged("model_3.2_lora_4bits", tokenizer, save_method = "merged_16bit",)


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions_2")
tokenizer.save_pretrained("/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions_2")

('/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions_2/tokenizer_config.json',
 '/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions_2/special_tokens_map.json',
 '/content/drive/MyDrive/Mestrado/Aula_IA024/Projeto_final/Models/llama_3.2_FT_lora_4000_questions_2/tokenizer.json')

# Load Model

In [ ]:
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model_path = "../Models/llama_3.2_FT_lora_2000_questions"
# model_path = "model_3.2_lora_4bits"

# Carregar o modelo e o tokenizador separadamente
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype = dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2024.10.6: Fast Llama patching. Transformers = 4.46.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU. Max memory: 3.712 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.10.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

# Test Loaded Model

In [ ]:
from unsloth.chat_templates import get_chat_template

question = dataset[5]['conversations'][0]['value']

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Which decoding algorithm for low-density parity-check codes involves a complex non-linear function in the check-node processing?\nOptions:\noption 1: Normalized min-sum\noption 2: Off-set min-sum\noption 3: Belief propagation\noption 4: Successive cancellation\noption 5: Sequential decoding\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer: option 3: Belief propagation\nExplanation: The belief propagation algorithm for low-density parity-check codes involves a complex non-linear function in the check-node processing, specifically the Q-function.<|eot_id|>']